In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

### Create a function that brings us the url

In [28]:
def get_url(position):
    template = 'https://mx.indeed.com/jobs?q={}&l='
    url = template.format(position)
    return url

In [29]:
url = get_url('desarrollador+web')

In [30]:
##Extract raw html

response = requests.get(url)
response

<Response [200]>

In [31]:
soup = BeautifulSoup(response.text,'html.parser')

In [32]:
cards = soup.find_all('div','jobsearch-SerpJobCard')

In [33]:
len(cards)

15

### Prototype the model with single record

In [18]:
card = cards[0]

In [19]:
atag = card.h2.a

In [20]:
job_title = atag.get('title')
job_title

'Desarrollador Back-end'

In [21]:
job_url = 'https://mx.indeed.com' + atag.get('href')
job_url

'https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CtmNdXQ2jCRnYz3MNwlgmu5VmVyG8nfi1qv3ecxLxLktQ2j3TX0n5nWYD97d_4m8rQ10xoAeOEe73yaAE5r4MuRA3dHALm15uUxKH5ZEetR9UoMtS9ZODZXgj48X9f1ZhsrwLlv25e12vWKD_jrIYo8KulWChv2Xjys4xySb9eH-9J8Qo_UonCIWbz8FOYbCf4rMGh_rkRkbT1AXp6SFj0hxvmIfeGVS15jl5iAy35HOSvxr9MP_oBj5IKZn9W842aLw5yCfQTC32U8u1k-gLVZ1si-qVpTnRecpOzqBDP6KH1lLhucqsLNpigTl2uTg32huQQAZfS98ZkzNfhzDF-4PY2f2_W2WDXPXsi6WIEzuDo8eDmNzAW3yPc7OMyRw-20kMGygDr1M0JkgXVR_Yg4Q96P71BitrpkUB9vx89B0oW56lcTySpKfVWqrVO8P56OFMmKiiWDXoiCSIK10wMnACSz57EfMo=&p=0&fvj=1&vjs=3'

In [22]:
company = card.find('span','company').text.strip()
company

'IPCom'

In [23]:
job_location = card.find('div','recJobLoc').get('data-rc-loc')
job_location

'Benito Juárez, CDMX'

In [35]:
job_summary = card.find('div','summary').text.strip().replace('\n',' ')

'Únete al mejor equipo de trabajo y vive la “Experiencia Play”. Tendrás oportunidad de desarrollar una carrera en el mejor casino de México.'

In [36]:
post_date = card.find('span','date').text

In [37]:
today = datetime.today().strftime('%Y-%m-%d')

'2021-01-24'

In [39]:
try:
    job_salaray = card.find('span','salaryText').text.strip()
except AttributeError:
    job_salary = ''

### Create master function

In [34]:
def get_record(card):
    atag = card.h2.a
    job_title = atag.get('title')
    job_url = 'https://mx.indeed.com' + atag.get('href')
    try:
        company = card.find('span','company').text.strip()
    except AttributeError:
        company = ''
    try:
        job_location = card.find('div','recJobLoc').get('data-rc-loc')
    except AttributeError:
        job_location = ''
    try:
        job_summary = card.find('div','summary').text.strip().replace('\n',' ')
    except AttributeError:
        job_summary = ''
    try:
        post_date = card.find('span','date').text
    except AttributeError:
        post_date = ''
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary = ''
        
    record = (job_title,company,job_location,post_date,today,job_summary,job_salary,job_url)
    
    return record

In [ ]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)
    
records

### How to obtain the next page?

In [36]:
while True:
    try:
        url = 'https://mx.indeed.com' + soup.find('a', {'aria-label': "Siguiente »"}).get('href')
    except AttributeError:
        break
        
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'jobsearch-SerpJobCard')
    
    for card in cards:
        record = get_record(card)
        records.append(record)

In [37]:
len(records)

750

In [38]:
with open('desarrollador+web.csv', 'w', newline = '', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title','Company','Location','Postdate','ExtractDate','Summary','Salary','JobURL'])
    writer.writerows(records)